### Author : Fathnelle Mehouelley

In [40]:
# import 
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import requests
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, to_date, year
from pathlib import Path
import gzip
import shutil
from pyspark.sql.types import StructField, StringType, StructType, DoubleType, IntegerType
from pyspark.sql.functions import input_file_name, lit
from pyspark.sql.functions import (
    col, year, month, weekofyear, avg, when, lag,
    to_date, date_format, to_timestamp, datediff, floor, lit
)
from pyspark.sql.window import Window
from pyspark.sql import functions as F
import geopandas as gpd
from pyspark.sql.functions import substring, avg as spark_avg
import folium
from folium import plugins
import json
from pyspark.ml.feature import VectorAssembler, StandardScaler
from pyspark.ml.regression import LinearRegression, RandomForestRegressor
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml import Pipeline

In [2]:
spark = SparkSession.builder \
    .appName("GasPricesAnalysis") \
    .config("spark.driver.memory", "8g") \
    .config("spark.executor.memory", "4g") \
    .config("spark.sql.shuffle.partitions", "200") \
    .getOrCreate()

spark


Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
26/01/11 07:16:38 WARN Utils: Your hostname, ENSAI-MSD47, resolves to a loopback address: 127.0.1.1; using 172.22.2.53 instead (on interface wlp0s20f3)
26/01/11 07:16:38 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/01/11 07:16:39 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# Data Collecting

### A Python Script that download files from Github to a directory provided to the function 

In [3]:
def download_gas_prices_data(output_dir="../data/raw", force_redownload=False):
    """
    Download gas price data from GitHub repository.
    
    Parameters:
    -----------
    output_dir : str
        Directory where files will be saved
    force_redownload : bool
        If True, redownload even if file already exists
    """
    # Base URL for raw files on GitHub
    BASE_URL = "https://raw.githubusercontent.com/rvm-courses/GasPrices/master/"
    
    # List of files to download
    FILES = [
        "Prix2022S1.csv.gz",
        "Prix2022S2.csv.gz",
        "Prix2023.csv.gz",
        "Prix2024.csv.gz",
        "Stations2024.csv.gz",
        "Services2024.csv.gz"
    ]
    
    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)
    
    print(f" Output directory: {output_dir}")
    print("=" * 70)
    
    success_count = 0
    failed_count = 0
    
    for filename in FILES:
        url = BASE_URL + filename
        file_path = os.path.join(output_dir, filename)
        
        # Check if file already exists
        if os.path.exists(file_path) and not force_redownload:
            file_size = os.path.getsize(file_path) / (1024 * 1024)
            print(f" {filename:25s} - Already exists ({file_size:.2f} MB)")
            success_count += 1
            continue
        
        try:
            print(f" {filename:25s} - Downloading...", end=" ", flush=True)
            response = requests.get(url, timeout=60)
            response.raise_for_status()
            
            # Save file
            with open(file_path, "wb") as f:
                f.write(response.content)
            
            file_size = len(response.content) / (1024 * 1024)
            print(f" ({file_size:.2f} MB)")
            success_count += 1
            
        except Exception as e:
            print(f" Error: {str(e)}")
            failed_count += 1
    
    print("=" * 70)
    print(f" Download completed! Success: {success_count} | Failed: {failed_count}")
    
    # List downloaded files
    downloaded_files = [f for f in os.listdir(output_dir) if f.endswith('.gz')]
    print(f" Total .gz files in directory: {len(downloaded_files)}")
    
    return output_dir


# Execute download
data_dir = download_gas_prices_data()
print(f"\n Files available in: {data_dir}")

 Output directory: ../data/raw
 Prix2022S1.csv.gz         - Downloading...  (15.59 MB)
 Prix2022S2.csv.gz         - Downloading...  (13.88 MB)
 Prix2023.csv.gz           - Downloading...  (23.50 MB)
 Prix2024.csv.gz           - Downloading...  (22.41 MB)
 Stations2024.csv.gz       - Downloading...  (0.35 MB)
 Services2024.csv.gz       - Downloading...  (0.24 MB)
 Download completed! Success: 6 | Failed: 0
 Total .gz files in directory: 6

 Files available in: ../data/raw


# Data Preparing

In [ ]:
# # Loading of data with Spark

# prices_2022_p1 = spark.read.option("header", True).option("inferSchema", True) \
#     .csv("../data/raw/Prix2022S1.csv")

# prices_2022_p2 = spark.read.option("header", True).option("inferSchema", True) \
#     .csv("../data/raw/Prix2022S2.csv")

# prices_2023 = spark.read.option("header", True).option("inferSchema", True) \
#     .csv("../data/raw/Prix2023.csv")

# prices_2024 = spark.read.option("header", True).option("inferSchema", True) \
#     .csv("../data/raw/Prix2024.csv")

# Stations_2024 = spark.read.option("header", True).option("inferSchema", True)\
#     .csv("../data/raw/Stations2024.csv")

# Services_2024 = spark.read.option("header", True).option("inferSchema", True) \
#     .csv("../data/raw/Services2024.csv")

26/01/11 07:17:55 WARN FileStreamSink: Assume no metadata directory. Error while looking for metadata directory in the path: ../data/raw/Prix2022S1.csv.
java.io.FileNotFoundException: File ../data/raw/Prix2022S1.csv does not exist
	at org.apache.hadoop.fs.RawLocalFileSystem.deprecatedGetFileStatus(RawLocalFileSystem.java:980)
	at org.apache.hadoop.fs.RawLocalFileSystem.getFileLinkStatusInternal(RawLocalFileSystem.java:1301)
	at org.apache.hadoop.fs.RawLocalFileSystem.getFileStatus(RawLocalFileSystem.java:970)
	at org.apache.hadoop.fs.FilterFileSystem.getFileStatus(FilterFileSystem.java:462)
	at org.apache.spark.sql.execution.streaming.sinks.FileStreamSink$.hasMetadata(FileStreamSink.scala:58)
	at org.apache.spark.sql.execution.datasources.DataSource.resolveRelation(DataSource.scala:384)
	at org.apache.spark.sql.catalyst.analysis.ResolveDataSource.org$apache$spark$sql$catalyst$analysis$ResolveDataSource$$loadV1BatchSource(ResolveDataSource.scala:143)
	at org.apache.spark.sql.catalyst.an

AnalysisException: [PATH_NOT_FOUND] Path does not exist: file:/home/ensai/Documents/MSD/Spark/Gas-Consumption-in-France-2025-2026/data/raw/Prix2022S1.csv. SQLSTATE: 42K03

In [6]:
# merging of gas files

# here , we use variables information available in the github
schema = StructType([
    StructField("id_pdv", StringType(), True),
    StructField("cp", StringType(), True),
    StructField("pop", StringType(), True),
    StructField("latitude", DoubleType(), True),
    StructField("longitude", DoubleType(), True),
    StructField("date", StringType(), True),
    StructField("id_carburant", StringType(), True),
    StructField("nom_carburant", StringType(), True),
    StructField("prix", DoubleType(), True)
])


# Directory containing the downloaded files
data_dir = "../data/raw"

# List of price files to read
price_files = [
    "Prix2022S1.csv.gz",
    "Prix2022S2.csv.gz",
    "Prix2023.csv.gz",
    "Prix2024.csv.gz"
]

# Read and merge all price files
dfs = []

for file in price_files:
    file_path = os.path.join(data_dir, file)
    
    print(f"Reading {file}...")
    
    # Read CSV with specified options
    df = spark.read \
        .option("header", "false") \
        .option("delimiter", ";") \
        .option("inferSchema", "false") \
        .schema(schema) \
        .csv(file_path)
    
    dfs.append(df)
    print(f" Rows: {df.count()}")

# Union all dataframes
print("\n Merging all files...")
gas = dfs[0]
for df in dfs[1:]:
    gas = gas.union(df)

print(f" Total rows after merge: {gas.count()}")

# Show sample data
print("\n Sample data:")
gas.show(10, truncate=False)

# Show schema
print("\n Schema:")
gas.printSchema()


Reading Prix2022S1.csv.gz...
 Rows: 2630689
Reading Prix2022S2.csv.gz...
 Rows: 2304432
Reading Prix2023.csv.gz...


 Rows: 4687530
Reading Prix2024.csv.gz...


 Rows: 4592186

 Merging all files...


 Total rows after merge: 14214837

 Sample data:
+-------+-----+---+---------+---------+-------------------+------------+-------------+-----+
|id_pdv |cp   |pop|latitude |longitude|date               |id_carburant|nom_carburant|prix |
+-------+-----+---+---------+---------+-------------------+------------+-------------+-----+
|1000001|01000|R  |4620100.0|519800.0 |2022-01-03T08:44:18|1           |Gazole       |1.572|
|1000001|01000|R  |4620100.0|519800.0 |2022-01-05T10:08:11|1           |Gazole       |1.56 |
|1000001|01000|R  |4620100.0|519800.0 |2022-01-10T10:09:08|1           |Gazole       |1.603|
|1000001|01000|R  |4620100.0|519800.0 |2022-01-17T12:34:42|1           |Gazole       |1.616|
|1000001|01000|R  |4620100.0|519800.0 |2022-01-17T12:34:44|1           |Gazole       |1.616|
|1000001|01000|R  |4620100.0|519800.0 |2022-01-17T12:34:45|1           |Gazole       |1.616|
|1000001|01000|R  |4620100.0|519800.0 |2022-01-21T09:46:30|1           |Gazole       |1.652|
|1000001|01000|R  |46

In [8]:
# spliting of the date 

# Convert date string to date type and extract components
gas = gas.withColumn("date_parsed", to_timestamp(col("date"), "yyyy-MM-dd'T'HH:mm:ss"))

# Extract year, month, and week of year
gas = gas.withColumn("year", year(col("date_parsed"))) \
         .withColumn("month", month(col("date_parsed"))) \
         .withColumn("week_of_year", weekofyear(col("date_parsed")))


# Preparing of latitude and longitude for mapping
# Checking of current values to determine the right power of 10
print("\nSample of current lat/lon values:")
gas.select("latitude", "longitude").show(5)

# Dividing by 100000 (10^5) to get proper coordinates

gas = gas.withColumn("latitude_corrected", col("latitude") / 100000) \
         .withColumn("longitude_corrected", col("longitude") / 100000)

print("\nCorrected coordinates added: latitude_corrected, longitude_corrected")

# Showing sample of corrected data
print("\n Sample of corrected coordinates:")
gas.select("latitude", "latitude_corrected", "longitude", "longitude_corrected").show(5)

# Final Schema and Sample
print("\n" + "=" * 70)
print(" FINAL SCHEMA")
print("=" * 70)
gas.printSchema()

print("\n" + "=" * 70)
print(" SAMPLE OF PROCESSED DATA")
print("=" * 70)
gas.select(
    "id_pdv", "date", "year", "month", "week_of_year",
    "latitude_corrected", "longitude_corrected", "nom_carburant", "prix"
).show(10, truncate=False)


Sample of current lat/lon values:
+---------+---------+
| latitude|longitude|
+---------+---------+
|4620100.0| 519800.0|
|4620100.0| 519800.0|
|4620100.0| 519800.0|
|4620100.0| 519800.0|
|4620100.0| 519800.0|
+---------+---------+
only showing top 5 rows

Corrected coordinates added: latitude_corrected, longitude_corrected

 Sample of corrected coordinates:
+---------+------------------+---------+-------------------+
| latitude|latitude_corrected|longitude|longitude_corrected|
+---------+------------------+---------+-------------------+
|4620100.0|            46.201| 519800.0|              5.198|
|4620100.0|            46.201| 519800.0|              5.198|
|4620100.0|            46.201| 519800.0|              5.198|
|4620100.0|            46.201| 519800.0|              5.198|
|4620100.0|            46.201| 519800.0|              5.198|
+---------+------------------+---------+-------------------+
only showing top 5 rows

 FINAL SCHEMA
root
 |-- id_pdv: string (nullable = true)
 |-- cp

With informations available in the github, we know that data is concerning France. When we look at France in a map :
Latitude (North-South) : between 41° et 51°
Longitude (East-West ) : between -5° et 9°
So we should divid latitude and longitude by 100000 (10^5). This is why we divided by 10^5 in the code 

In [9]:
# Creating of the Spark SQL table 

gas.createOrReplaceTempView("gas_prices")
spark.sql("SELECT COUNT(*) as total_rows FROM gas_prices").show()


+----------+
|total_rows|
+----------+
|  14214837|
+----------+



In [ ]:
# Basics Statistics 

# Counting records by gas type
print("\n Number of records per gas type:")
spark.sql("""
    SELECT nom_carburant, 
           COUNT(*) as count,
           ROUND(COUNT(*) * 100.0 / SUM(COUNT(*)) OVER(), 2) as percentage
    FROM gas_prices
    GROUP BY nom_carburant
    ORDER BY count DESC
""").show()

# we have 6 types of gas

# Average price by gas type
print("\n Average price by gas type:")
spark.sql("""
    SELECT nom_carburant,
           COUNT(*) as records,
           ROUND(AVG(prix), 2) as avg_price,
           ROUND(MIN(prix), 2) as min_price,
           ROUND(MAX(prix), 2) as max_price
    FROM gas_prices
    GROUP BY nom_carburant
    ORDER BY records DESC
""").show()

# Number of stations selling each gas type
print("\n Number of stations per gas type:")
spark.sql("""
    SELECT nom_carburant,
    COUNT(DISTINCT id_pdv) as num_stations
    FROM gas_prices
    GROUP BY nom_carburant
    ORDER BY num_stations DESC
""").show()


 Number of records per gas type:


26/01/11 07:23:57 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
26/01/11 07:23:57 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
26/01/11 07:23:57 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
26/01/11 07:24:00 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
26/01/11 07:24:00 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
26/01/11 07:24:00 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
26/01/11 0

+-------------+-------+----------+
|nom_carburant|  count|percentage|
+-------------+-------+----------+
|       Gazole|4245380|     29.87|
|          E10|3559498|     25.04|
|         SP98|3425844|     24.10|
|          E85|1390580|      9.78|
|         SP95| 961020|      6.76|
|         GPLc| 619759|      4.36|
|         NULL|  12756|      0.09|
+-------------+-------+----------+


 Average price by gas type:


+-------------+-------+---------+---------+---------+
|nom_carburant|records|avg_price|min_price|max_price|
+-------------+-------+---------+---------+---------+
|       Gazole|4245380|     1.81|      0.0|     10.0|
|          E10|3559498|     1.82|      0.0|     10.0|
|         SP98|3425844|     1.91|      0.0|     10.0|
|          E85|1390580|     0.92|      0.0|     9.99|
|         SP95| 961020|     1.85|      0.0|     10.0|
|         GPLc| 619759|     0.96|     0.02|      7.0|
|         NULL|  12756|     NULL|     NULL|     NULL|
+-------------+-------+---------+---------+---------+


 Number of stations per gas type:


+-------------+------------+
|nom_carburant|num_stations|
+-------------+------------+
|       Gazole|       10391|
|         SP98|        8873|
|          E10|        8256|
|         SP95|        5049|
|         NULL|        5019|
|          E85|        4067|
|         GPLc|        1730|
+-------------+------------+



we can filtered out NULL and GLPc cause there have the lowest number of record values

In [ ]:
# Filtering out of 'NULL' and 'GPLc'

gas_filtered = spark.sql("""
    SELECT *
    FROM gas_prices
    WHERE nom_carburant NOT IN ('NULL', 'GPLc')
""")

print("\nOriginal record count:")
spark.sql("SELECT COUNT(*) as count FROM gas_prices").show()

print("\nFiltered record count:")
gas_filtered.createOrReplaceTempView("gas_filtered")
spark.sql("SELECT COUNT(*) as count FROM gas_filtered").show()

print("\nRemaining gas types:")
spark.sql("""
    SELECT nom_carburant, COUNT(*) as count
    FROM gas_filtered
    GROUP BY nom_carburant
    ORDER BY count DESC
""").show()



Original record count:


+--------+
|   count|
+--------+
|14214837|
+--------+


Filtered record count:


+--------+
|   count|
+--------+
|13582322|
+--------+


Remaining gas types:


+-------------+-------+
|nom_carburant|  count|
+-------------+-------+
|       Gazole|4245380|
|          E10|3559498|
|         SP98|3425844|
|          E85|1390580|
|         SP95| 961020|
+-------------+-------+



In [13]:
gas_filtered.show()

+-------+-----+---+---------+---------+-------------------+------------+-------------+-----+-------------------+----+-----+------------+------------------+-------------------+
| id_pdv|   cp|pop| latitude|longitude|               date|id_carburant|nom_carburant| prix|        date_parsed|year|month|week_of_year|latitude_corrected|longitude_corrected|
+-------+-----+---+---------+---------+-------------------+------------+-------------+-----+-------------------+----+-----+------------+------------------+-------------------+
|1000001|01000|  R|4620100.0| 519800.0|2022-01-03T08:44:18|           1|       Gazole|1.572|2022-01-03 08:44:18|2022|    1|           1|            46.201|              5.198|
|1000001|01000|  R|4620100.0| 519800.0|2022-01-05T10:08:11|           1|       Gazole| 1.56|2022-01-05 10:08:11|2022|    1|           1|            46.201|              5.198|
|1000001|01000|  R|4620100.0| 519800.0|2022-01-10T10:09:08|           1|       Gazole|1.603|2022-01-10 10:09:08|2022|   

#### Computing of price index for each station per week

In [19]:

# Extracting date from timestamp for grouping
gas_filtered = gas_filtered.withColumn("date_only", col("date_parsed").cast("date"))

# Computing of the average daily price in France for each gas type
print(" Computing average daily prices in France")

avg_daily_prices = gas_filtered.groupBy("date_only", "nom_carburant") \
    .agg(avg("prix").alias("avg_day_price_france"))

print("Sample of average daily prices:")
avg_daily_prices.orderBy("date_only", "nom_carburant").show(10)

# Joining back to original data
print("\nStep 2: Joining average prices to station data...")

gas_with_avg = gas_filtered.join(
    avg_daily_prices,
    on=["date_only", "nom_carburant"],
    how="left"
)

# Computing of Price Index
print(" Computing Price Index...")

gas_with_index = gas_with_avg.withColumn(
    "price_index",
    (((col("prix") - col("avg_day_price_france")) / col("avg_day_price_france")) + 1 ) * 100
)


# Display results
print("\nSample of data with Price Index:")
gas_with_index.select(
    "id_pdv", "date_only", "nom_carburant", 
    "prix", "avg_day_price_france", "price_index"
).orderBy("date_only").show(20)

# Statistics on Price Index
print("\nPrice Index statistics:")
gas_with_index.groupBy("nom_carburant") \
    .agg(
        F.avg("price_index").alias("avg_index"),
        F.min("price_index").alias("min_index"),
        F.max("price_index").alias("max_index")
    ).show()

# Update the filtered gas dataframe
gas_prices = gas_with_index

print("\nPrice Index computation completed")


 Computing average daily prices in France
Sample of average daily prices:


+----------+-------------+--------------------+
| date_only|nom_carburant|avg_day_price_france|
+----------+-------------+--------------------+
|2022-01-01|          E10|  1.6803428869202677|
|2022-01-01|          E85|   0.750264884568652|
|2022-01-01|       Gazole|   1.613302611367121|
|2022-01-01|         SP95|  1.6803859649122803|
|2022-01-01|         SP98|  1.7814656084656062|
|2022-01-02|          E10|  1.6753885267275093|
|2022-01-02|          E85|  0.7530445945945937|
|2022-01-02|       Gazole|  1.6055162676488615|
|2022-01-02|         SP95|  1.6604820143884889|
|2022-01-02|         SP98|  1.7746308681672003|
+----------+-------------+--------------------+
only showing top 10 rows

Step 2: Joining average prices to station data...
 Computing Price Index...

Sample of data with Price Index:


+--------+----------+-------------+-----+--------------------+------------------+
|  id_pdv| date_only|nom_carburant| prix|avg_day_price_france|       price_index|
+--------+----------+-------------+-----+--------------------+------------------+
| 1000012|2022-01-01|       Gazole|1.662|   1.613302611367121|103.01849065945616|
| 1000012|2022-01-01|         SP98|1.824|  1.7814656084656062|102.38760666118213|
| 1000012|2022-01-01|          E10|1.714|  1.6803428869202677|102.00299077894863|
| 2200014|2022-01-01|         SP98|1.817|  1.7814656084656062|101.99467176719735|
| 1170001|2022-01-01|         SP95|1.623|  1.6803859649122803| 96.58495333152369|
| 1000013|2022-01-01|         SP98|1.706|  1.7814656084656062|  95.7638470197241|
| 2200014|2022-01-01|          E10|1.707|  1.6803428869202677| 101.5864091363275|
| 1120008|2022-01-01|         SP98|1.879|  1.7814656084656062|105.47495225677702|
| 2200014|2022-01-01|       Gazole|1.647|   1.613302611367121|102.08872088816143|
| 1130003|2022-0

+-------------+------------------+-------------------+------------------+
|nom_carburant|         avg_index|          min_index|         max_index|
+-------------+------------------+-------------------+------------------+
|          E10|100.00000000000189| 0.0584636146616857| 588.7343904794254|
|         SP98|100.00000000000301|0.05515107431606214| 555.0886815602645|
|          E85|100.00000000000107|0.11508514816558346|1087.7875329825256|
|       Gazole|100.00000000000068|0.05097669940030691|  597.972892493076|
|         SP95|100.00000000000003|0.19557074495560878| 521.8188398646778|
+-------------+------------------+-------------------+------------------+


Price Index computation completed


#### Computing of week index

In [24]:
# we find the first date in the entire dataset

gas_prices = gas_prices.withColumn("date_only", col("date_parsed").cast("date"))
gas_prices.show(5)
first_date = gas_prices.select(F.min("date_only")).collect()[0][0]

first_date = gas_prices.select(F.min("date_only")).collect()[0][0]
print(f"First date in dataset: {first_date}")

# Computing of week index
gas_prices = gas_prices.withColumn(
    "week_index",
    floor(datediff(col("date_only"), lit(first_date)) / 7) + 1
)

# Verification of the results
print("\nSample of data with week index:")
gas_prices.select(
    "date_only", "year", "week_of_year", "week_index"
).orderBy("date_only").show(20)

print("\nWeek index statistics:")
gas_prices.select(
    F.min("week_index").alias("min_week"),
    F.max("week_index").alias("max_week")
).show()

print("\nRecords per week index (first 10 weeks):")
gas_prices.groupBy("week_index") \
    .count() \
    .orderBy("week_index") \
    .show(10)
print("\nWeek Index computation completed")

+----------+-------------+-------+-----+---+---------+---------+-------------------+------------+-----+-------------------+----+-----+------------+------------------+-------------------+--------------------+-----------------+----------+
| date_only|nom_carburant| id_pdv|   cp|pop| latitude|longitude|               date|id_carburant| prix|        date_parsed|year|month|week_of_year|latitude_corrected|longitude_corrected|avg_day_price_france|      price_index|week_index|
+----------+-------------+-------+-----+---+---------+---------+-------------------+------------+-----+-------------------+----+-----+------------+------------------+-------------------+--------------------+-----------------+----------+
|2022-01-17|       Gazole|1000001|01000|  R|4620100.0| 519800.0|2022-01-17T12:34:42|           1|1.616|2022-01-17 12:34:42|2022|    1|           3|            46.201|              5.198|  1.6495400853890057|97.96670079823517|         3|
|2022-01-17|       Gazole|1000001|01000|  R|4620100.

First date in dataset: 2022-01-01

Sample of data with week index:


+----------+----+------------+----------+
| date_only|year|week_of_year|week_index|
+----------+----+------------+----------+
|2022-01-01|2022|          52|         1|
|2022-01-01|2022|          52|         1|
|2022-01-01|2022|          52|         1|
|2022-01-01|2022|          52|         1|
|2022-01-01|2022|          52|         1|
|2022-01-01|2022|          52|         1|
|2022-01-01|2022|          52|         1|
|2022-01-01|2022|          52|         1|
|2022-01-01|2022|          52|         1|
|2022-01-01|2022|          52|         1|
|2022-01-01|2022|          52|         1|
|2022-01-01|2022|          52|         1|
|2022-01-01|2022|          52|         1|
|2022-01-01|2022|          52|         1|
|2022-01-01|2022|          52|         1|
|2022-01-01|2022|          52|         1|
|2022-01-01|2022|          52|         1|
|2022-01-01|2022|          52|         1|
|2022-01-01|2022|          52|         1|
|2022-01-01|2022|          52|         1|
+----------+----+------------+----

+--------+--------+
|min_week|max_week|
+--------+--------+
|       1|     157|
+--------+--------+


Records per week index (first 10 weeks):


+----------+------+
|week_index| count|
+----------+------+
|         1| 91529|
|         2| 92117|
|         3| 91260|
|         4| 83414|
|         5| 95034|
|         6| 93617|
|         7| 92613|
|         8| 97797|
|         9|123185|
|        10|136597|
+----------+------+
only showing top 10 rows

Week Index computation completed


In [25]:
# Saving as CSV in data/processed

output_dir = "../data/processed"
gas_prices.coalesce(1) \
    .write \
    .mode("overwrite") \
    .option("header", "true") \
    .option("delimiter", ",") \
    .csv(output_dir + "/gas_prices_processed")

print(f"Data saved to {output_dir}/gas_prices")

Data saved to ../data/processed/gas_prices


In [26]:
output_dir = "../data/processed"
data_dir = "../data/raw"

# Stations schema
# from the github, we have this information about variable in stations dataset
stations_schema = StructType([
    StructField("id_pdv", StringType(), True),
    StructField("cp", StringType(), True),
    StructField("pop", StringType(), True),
    StructField("latitude", DoubleType(), True),
    StructField("longitude", DoubleType(), True),
    StructField("adresse", StringType(), True),
    StructField("ville", StringType(), True)
])

# Reading and saving of Stations file
print("Processing Stations")
stations_df = spark.read \
    .option("header", "false") \
    .option("delimiter", "|") \
    .schema(stations_schema) \
    .csv(f"{data_dir}/Stations2024.csv.gz")

stations_df.write \
    .mode("overwrite") \
    .option("header", "true") \
    .csv(f"{output_dir}/Stations2024")

print(f"Stations saved to {output_dir}/Stations2024/")

# Services schema
services_schema = StructType([
    StructField("id_pdv", StringType(), True),
    StructField("cp_pdv", StringType(), True),
    StructField("pop", StringType(), True),
    StructField("latitude", DoubleType(), True),
    StructField("longitude", DoubleType(), True),
    StructField("services", StringType(), True)
])

# Read and save Services
print("Processing Services...")
services_df = spark.read \
    .option("header", "false") \
    .option("delimiter", "|") \
    .schema(services_schema) \
    .csv(f"{data_dir}/Services2024.csv.gz")

services_df.write \
    .mode("overwrite") \
    .option("header", "true") \
    .csv(f"{output_dir}/Services2024")

print(f"Services saved to {output_dir}/Services2024/")

print("\nAll files processed!")

Processing Stations
Stations saved to ../data/processed/Stations2024/
Processing Services...
Services saved to ../data/processed/Services2024/

All files processed!


# Visualization of gas prices

In [35]:
# we are going to do an interactive visualization of gas prices
print("=" * 70)
print("      Interactive visualization with plotly ")
print("=" * 70)

# Computing of weekly average price
weekly_avg = gas_prices.groupBy("week_index", "nom_carburant") \
    .agg(avg("prix").alias("avg_price")) \
    .orderBy("week_index", "nom_carburant")

df_viz = weekly_avg.toPandas()

# Creating of interactive plot
fig = px.line(
    df_viz,
    x="week_index",
    y="avg_price",
    color="nom_carburant",
    markers=True,
    title="Weekly Evolution of Average Gas Prices in France",
    labels={
        "week_index": "Week Index",
        "avg_price": "Average Price (euros)",
        "nom_carburant": "Gas Type"
    }
)

fig.update_layout(
    hovermode="x unified",
    width=1200,
    height=600,
    font=dict(size=12)
)

fig.show()

print("Visualization completed!")

      Interactive visualization with plotly 


Visualization completed!


### Bonus question 

In [27]:
# Install folium package
!pip install folium

9349.44s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [folium]


In [28]:

!pip install geopandas

9364.30s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.4/32.4 MB 3.3 MB/s  0:00:09m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 3.0 MB/s  0:00:03m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 3.7 MB/s  0:00:00 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4/4 [geopandas]/4 [geopandas]


In [ ]:
print("=" * 70)
print("        Geographic Heat Maps od prices indexes by department ")
print("=" * 70)

# Computing of average price index by department and gas type
# we are extracting department code from postal code (first 2 digits)

gas_with_dept = gas_prices.withColumn(
    "department", 
    substring(col("cp"), 1, 2)
)

# Computing of average price index by department and gas type
dept_avg = gas_with_dept.groupBy("department", "nom_carburant") \
    .agg(spark_avg("price_index").alias("avg_price_index")) \
    .orderBy("department", "nom_carburant")

# Converting to Pandas
df_dept = dept_avg.toPandas()

print(f"Departments: {df_dept['department'].nunique()}")
print(f"Gas types: {df_dept['nom_carburant'].nunique()}")
print("\nSample data:")
print(df_dept.head())

# now , we ere downloading French departments GeoJSON
print("\n Downloading French departments GeoJSON...")

geojson_url = "https://raw.githubusercontent.com/gregoiredavid/france-geojson/master/departements.geojson"
response = requests.get(geojson_url)
france_geo = response.json()

print("GeoJSON downloaded successfully")

# Creating of maps for each gas type

gas_types = df_dept["nom_carburant"].unique()

for gas_type in gas_types:
    print(f"\nCreating map for {gas_type}...")
    
    # Filtering data for this gas type
    gas_data = df_dept[df_dept["nom_carburant"] == gas_type].copy()
    
    # Create base map centered on France
    m = folium.Map(
        location=[46.603354, 1.888334],
        zoom_start=6,
        tiles='OpenStreetMap'
    )
    
    # Creating of choropleth map
    folium.Choropleth(
        geo_data=france_geo,
        name='choropleth',
        data=gas_data,
        columns=['department', 'avg_price_index'],
        key_on='feature.properties.code',
        fill_color='YlOrRd',
        fill_opacity=0.7,
        line_opacity=0.2,
        legend_name=f'Average Price Index - {gas_type}',
        nan_fill_color='lightgray'
    ).add_to(m)
    
    # Adding of title
    title_html = f'''
    <div style="position: fixed; 
                top: 10px; left: 50px; width: 400px; height: 60px; 
                background-color: white; border:2px solid grey; z-index:9999; 
                font-size:16px; font-weight: bold; padding: 10px">
        <p style="margin: 0;">Price Index Heat Map - {gas_type}</p>
        <p style="margin: 0; font-size: 12px; font-weight: normal;">France - By Department</p>
    </div>
    '''
    m.get_root().html.add_child(folium.Element(title_html))
    
    # Saving of the  map
    output_file = f"../data/processed/heatmap_{gas_type.replace(' ', '_')}.html"
    m.save(output_file)
    print(f"Map saved to {output_file}")

print("\n" + "=" * 70)
print("ALL HEAT MAPS CREATED SUCCESSFULLY")
print("=" * 70)

GEOGRAPHIC HEAT MAPS OF PRICE INDEXES BY DEPARTMENT

1. Computing average price index by department...


Departments: 95
Gas types: 5

Sample data:
  department nom_carburant  avg_price_index
0         01           E10       100.710158
1         01           E85       101.420025
2         01        Gazole       100.488688
3         01          SP95        99.940917
4         01          SP98       100.802855

2. Downloading French departments GeoJSON...
GeoJSON downloaded successfully

3. Creating heat maps...

Creating map for E10...
Map saved to ../data/processed/heatmap_E10.html

Creating map for E85...
Map saved to ../data/processed/heatmap_E85.html

Creating map for Gazole...
Map saved to ../data/processed/heatmap_Gazole.html

Creating map for SP95...
Map saved to ../data/processed/heatmap_SP95.html

Creating map for SP98...
Map saved to ../data/processed/heatmap_SP98.html

ALL HEAT MAPS CREATED SUCCESSFULLY


# Modeling gas prices evolution

#### We are going to use two models : LinearRegression and RandomForestRegression to forecast the next price day of gas

In [ ]:
# We create lag features
